In [19]:
#基于内容过滤（CBF）、协同过滤（CF）
#效用矩阵：表示用户喜欢商品的程度的数据矩阵

import numpy as np
import pandas as pd
import copy
import collections
from scipy import linalg
import math
from collections import defaultdict
#导入商品数据
df = pd.read_csv('./data/ml-100k/u.data',encoding='ISO-8859-1',sep='\t',header=None)
df_info = pd.read_csv('./data/ml-100k/u.item',encoding='ISO-8859-1',sep='|',header=None)
movielist = [df_info[1].tolist()[indx]+';'+str(indx+1) for indx in range(len(df_info[1].tolist()))]
nmovies = len(movielist)
nusers = len(df[0].drop_duplicates().tolist())#去重并转为list
min_ratings = 50
movies_rated  = list(df[1]) 
counts = collections.Counter(movies_rated)
print(counts)
print('ratings less than ',min_ratings,':',len(collections.Counter(el for el in counts.elements() if counts[el] <min_ratings)))
dfout = pd.DataFrame(columns=['user']+movielist)
toremovelist = []
for i in range(1,nusers):
    tmpmovielist = [0 for j in range(nmovies)]
    dftmp =df[df[0]==i]
    for k in dftmp.index:
        if counts[dftmp.loc[k][1]]>= min_ratings:           
           tmpmovielist[dftmp.loc[k][1]-1] = dftmp.loc[k][2]
        else:
           toremovelist.append(dftmp.loc[k][1])
            
    dfout.loc[i] = [i]+tmpmovielist
  
toremovelist = list(set(toremovelist))
dfout.drop(dfout.columns[toremovelist], axis=1, inplace=True)
print(dfout.head())
dfout.to_csv('data/utilitymatrix.csv',index=None)
df = pd.read_csv('./data/utilitymatrix.csv')
df.head(2)
#0表示缺失值

Counter({50: 583, 258: 509, 100: 508, 181: 507, 294: 485, 286: 481, 288: 478, 1: 452, 300: 431, 121: 429, 174: 420, 127: 413, 56: 394, 7: 392, 98: 390, 237: 384, 117: 378, 172: 367, 222: 365, 204: 350, 313: 350, 405: 344, 79: 336, 210: 331, 151: 326, 173: 324, 69: 321, 748: 316, 168: 316, 269: 315, 257: 303, 195: 301, 423: 300, 9: 299, 318: 298, 276: 298, 302: 297, 22: 297, 328: 295, 96: 295, 118: 293, 15: 293, 25: 293, 183: 291, 216: 290, 176: 284, 64: 283, 234: 280, 202: 280, 191: 276, 28: 276, 89: 275, 111: 272, 275: 268, 12: 267, 742: 267, 357: 264, 82: 261, 135: 259, 289: 259, 97: 256, 238: 256, 268: 255, 546: 254, 196: 251, 333: 251, 186: 251, 70: 251, 475: 250, 153: 247, 132: 246, 228: 244, 125: 244, 144: 243, 483: 243, 194: 241, 245: 240, 323: 240, 197: 239, 185: 239, 11: 236, 282: 232, 496: 231, 568: 230, 301: 230, 265: 227, 655: 227, 182: 226, 273: 223, 143: 222, 471: 221, 179: 221, 180: 221, 71: 220, 161: 220, 95: 219, 427: 219, 678: 219, 8: 219, 322: 218, 235: 217, 435: 216

  user Toy Story (1995);1 GoldenEye (1995);2 Four Rooms (1995);3  \
1    1                  5                  3                   4   
2    2                  4                  0                   0   
3    3                  0                  0                   0   
4    4                  0                  0                   0   
5    5                  4                  3                   0   

  Get Shorty (1995);4 Copycat (1995);5 Twelve Monkeys (1995);7 Babe (1995);8  \
1                   3                3                       4             1   
2                   0                0                       0             0   
3                   0                0                       0             0   
4                   0                0                       0             0   
5                   0                0                       0             0   

  Dead Man Walking (1995);9 Richard III (1995);10  ...  \
1                         5                     3  .

,user,Toy Story (1995);1,GoldenEye (1995);2,Four Rooms (1995);3,Get Shorty (1995);4,Copycat (1995);5,Twelve Monkeys (1995);7,Babe (1995);8,Dead Man Walking (1995);9,Richard III (1995);10,...,Cool Runnings (1993);1035,Hamlet (1996);1039,Forget Paris (1995);1041,Multiplicity (1996);1047,She's the One (1996);1048,Koyaanisqatsi (1983);1065,Shallow Grave (1994);1073,Reality Bites (1994);1074,Six Degrees of Separation (1993);1101,Some Kind of Wonderful (1987);1119
0,1,5,3,4,3,3,4,1,5,3,...,0,0,0,0,0,0,0,0,0,0
1,2,4,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0


In [21]:
#对缺失值赋予初始值：用户给出的平均分或每件商品的平均分
def imputation(inp,Ri):
    Ri = Ri.astype(float)
    def userav():
        for i in range(len(Ri)):
            Ri[i][Ri[i]==0]=sum(Ri[i])/float(len(Ri[i][Ri[i]>0]))#将所有为0的缺失值赋值为用户的平均分
            return Ri
    def itemav():
        for i in range(len(Ri[0])):
            Ri[:,i][Ri[:,i]==0]=sum(Ri[:,i])/float(len(Ri[:,i][Ri[:,i]>0]))#缺失值赋值为每件商品的平均分
        return Ri
    switch = {'useraverage':userav(),'itemaverage':itemav()}
    return switch(inp)


In [23]:
#相似度度量方法:余弦相似度、皮尔逊相关系数，均值为0两种方法结果相同
#计算两个向量之间的相似度函数（下）
from scipy.stats import pearsonr
from scipy.spatial.distance import cosine
def sim(x,y,metrix='cos'):
    if metrix == 'cos':
        return 1.-cosine(x,y)
    else:
        return pearsonr(x,y)[0]


In [25]:
#基于用户的协同过滤
#使用KNN算法找出与给定用户打分记录相似的用户，对打分取加权平均值
#（1）用相似度度量方法找出相似用户（2）对于没有打分的商品使用打分的加权平均值
def CF_userbased(u_vec,K,data,index=False):
    def FindKNeighbours(r,data,K):
        neighs = []
        cnt = 0
        for u in range(len(data)):
            if data[u,r]>0 and cnt<K:
                neighs.append(data[u])
                cnt+=1
            elif cnt==X:
                break
        return np.array(neighs)
    def CalcRating(u_vec,r,neighs):    #计算预测值
        rating = 0
        den = 0
        for j in range(len(neighs)):
            rating += neighs[j][-1]*float(neighs[j][r]-neighs[j][neighs[j]>0][:-1].mean())
            den += abs(neighs[j][-1])
        if den>0:
            rating = np.round(u_vec[u_vec>0].mean()+(rating/den),0)
        else:
            rating = np.round(u_vec[u_vec>0].mean(),0)
        if rating>5:
            return 5
        elif rating<1:
            return 1
        return rating
    #添加相似列
    data = data.astype(float)
    nrows = len(data)
    ncols = len(data[0])
    data_sim = np.zeros((nrows,ncols+1))
    data_sim[:,:-1] = data
    
    for u in range(nrows):           #计算用户打的分数与
        if np.array_equal(data_sim[u,:-1],u_vec)==False:   
            data_sim[u,ncols] = sim(data_sim[u,:-1],u_vec,'pearson')
        else:
            data_sim[u,ncols] = 0.
    data_sim = data_sim[data_sim[:,ncols].argsort()][::-1]
    for r in range(ncols):
        if u_vec[r] == 0:
            neighs = FindKNeighbours(r,data_sim,K)
            u_vec[r] = CalcRating(u_vec,r,neighs)
    if indxs:
        seenindxs = [indx for indx in range(len(u_vec)) if u_vec[indx]>0]
        u_vec[seenindxs] = -1
        recsvec = np.argsort(u_vec)[::-1][np.argsort(u_vec)>0]
        
        return recsvec
    return u_vec

In [26]:
#基于商品的协同过滤
#（1）使用一种相似度度量方法，找出用户i打过分的商品最相似的K件商品
#计算K件商品分数加权平均值，作为预测值
class CF_itembased(object):
    def _init_(self,data):
        nitem = len(data[0])
        self.data = data
        self.simmatrix = np.zeros((nitems,nitems))
        for i in range(nitems):
            for j in range(nitems):
                if j>=1:
                    self.simmatrix[i,j] = sim(data[:,i],data[:,j])
                else:
                    self.simmatrix[i,j] = self.simmatrix[j,i]
    def GetKSimItemsperUser(self,r,K,u_vec):        #找到K个近邻的商品
        items = np.argsort(self.simmatrix[r])[::-1]
        items = items[items!=r]
        cnt = 0
        neighitems = []
        for i in items:
            if u_vec[i]>0 and cnd<K:
                neighitems.append(i)
                cnt+=1
            elif cnt==K:
                break
        return neighitems
    def CalcRating(self,r,u_vec,neighitems):
        rating = 0
        den = 0
        for i in neighitems:
            rating += self.simmatrix[r,i]*u_vec[i]
            den += abs(self.simmatrix[r,i])
        if den>0:
            rating = np.round(rating/den,0)
        else:
            rating = np.round(self.data[:,r][self.data[:,r]>0].mean(),0)
        return rating
    def CalcRatings(self,u_vec,K,indx = False):
        u_rec = np.zeros(len(u_vec))
        for r in range(len(u_vec)):
            if u_vec[r] == 0:
                neighitems = self.GetKSimItemsperUser(r,K,u_vec)
                u_rec[r] = self.CalcRatings(r,u_vec,neighitems)
        if indxs:
            seenindxs = [indx for indx in range(len(u_vec)) if u_vec[indx]>0]
            u_vec[seenindxs] = -1
            recsvec = np.argsort(u_rec)[::-1][np.argsort(u_rec)>0]
            
            return recsvec
        return u_rec
        

In [29]:
#slope one算法（最简单的基于商品的协同过滤）
#通过计算得到矩阵D，Dij表示商品i与商品j的平均差值
#（1）找出与商品j差值最小的K件商品，计算加权平均值作为预测值
class SlopeOne(object):
    def _init_(self,Umatrix):
        nitem = len(Umatrix[0])
        self.dif,matrix = np.zeros((nitems,nitems))
        self.nratings = np.zeros((nitems,nitems))
        def diffav_n(x,y):
            xy = np.vstack((x,y)).T
            xy = xy[(xy[:,0]>0)&(xy[:,1]>0)]
            nxy = len(xy)
            if nxy==0:
                return [1000.,0]
            return [float(sum(xy[:,0]))/nxy,nxy]
        for i in range(nitems):
            for j in range(nitems):
                if j>=1:
                    self.difmatrix[i,j],self.nratings[i,j] = diffac_n(Umatrix[:,i],Umatrix[:,j])
                else:
                    self.difmatrix[i,j] = -self.difmatrix[j,i]
                    self.nratings[i,j] = self.nrating[j,i]
                    
    def GetKSimItemsperUser(self,r,K,u_vec):        #找到K个近邻的商品
        items = np.argsort(self.difmatrix[r])
        items = items[items!=r]
        cnt = 0
        neighitems = []
        for i in items:
            if u_vec[i]>0 and cnd<K:
                neighitems.append(i)
                cnt+=1
            elif cnt==K:
                break
        return neighitems
    def CalcRating(self,r,u_vec,neighitems):
        rating = 0
        den = 0
        for i in neighitems:
            if abs(self.difmatrix[r,i])!=1000:
                rating += (self.difmatrix[r,i]+u_vec[i])*self.nratings[r,i]
                den += self.nratings[r,i]
        if den==0:
            return 0.
        rating = np.round(rating/den,0)
        if rating>5:
            return 5
        elif rating<1.:
            return 1.
        return rating
    def CalcRatings(self,u_vec,K):
        u_rec = np.zeros(len(u_vec))
        for r in range(len(u_vec)):
            if u_vec[r] == 0:
                neighitems = self.GetKSimItemsperUser(r,K,u_vec)
                u_rec[r] = self.CalcRating(r,u_vec,neighitems)
        return u_rec
        

In [31]:
#基于模型的协同过滤
#利用效用矩阵生成模型，抽取用户的打分模式。包含各种模式的模型返回预测值，填充或逼近原始矩阵（矩阵分解）
#交替最小二乘（ALS）

def ALS(Umatrix,K,iterations=50,l=0.001,tol=0.001):
    nrows = len(Umatrix)
    ncols = len(Umatrix[0])
    P = np.random.rand(nrows,K)
    Q = np.random.rand(ncols,K)
    Qt = Q.T
    err = 0.
    Umatrix = Umatrix.astype(float)
    mask = Umatrix>0
    mask[mask==True] = 1
    mask[mask==False] = 0
    mask = mask.astype(np.float64,copy = False)
    for it in range(iterations):
        for u,mask_u in enumerate(mask):  #矩阵的形式计算线性方程
            P[u] = np.linalg.solve(np.dot(Qt,np.dot(np.diag(mask_u),Qt.T))
                                   +l*np.eye(K),np.dot(Qt,np.dot(np.diag(mask_u),Umatrix[u].T))).T
        for i,mask_i in enumerate(mask.T):
            Qt[:,i]  =np.linalg.solve(np.dot(P.T,np.dot(np.diag(mask_i),P))
                                   +l*np.eye(K),np.dot(P.T,np.dot(np.diag(mask_i),Umatrix[:,i])))
        err = np.sum((mask*(Umatrix-np.dot(P,Qt)))**2)#不断迭代，直至误差小于设定值
        if err<tol:
            break
    return np.round(np.dot(P,Qt),0)
#随机梯度下降（SGD）,也属于矩阵分解类别
def SGD(Umatrix,K,iterations=100,alpha=0.00001,l=0.001,tol=0.001):
    nrows = len(Umatrix)
    ncols = len(Umatrix[0])
    P = np.random.rand(nrows,K)
    Q = np.random.rand(ncols,K)
    Qt = Q.T
    cost = -1
    for it in range(iterations):
        for j in range(ncols):
            if Umatrix[i][j]>0:
                eij = Umatrix[i][j]-np.dot(P[i,:],Qt[:,j])
                for k in range(K):
                    P[i][k] += alpha*(2*eij*Qt[k][j]-l*P[i][k])
                    Qt[k][j] += alpha*(2*eij*P[i][k]-l*Qt[k][j])
        cost = 0
        for i in range(nrows):
            for j in range(ncols):
                if Umatrix[i][j]>0:
                    cost += pow(Umatrix[i][j]-np.dot(P[i,:],Qt[:,j]),2)
                    for k in range(K):
                        cost+=float(1/2.0)*(pow(P[i][k],2)+pow(Qt[k][j],2))
        if cost<tol:
            break    
    return np.round(np.dot(P,Qt),0)
#非负矩阵分解（NMF）
#这里没有进行重写，直接调用sklearn库中的NMF函数
from sklearn.decomposition import NMF
def NMF_alg(Umtrix,K,inp='none',l=0.001):
    R_tmp = copy.copy(Umatrix)
    R_tmp = R_tmp.astype(float)
    if inp!='none':
        R_tmp = imputation(inp,Umatrix)#缺失值处理
    nmf = NMF(n_components=K,alpha=l)
    P = nmf.fit_transform(R_tmp)
    R_tmp = np.dot(P,nmf.components_)
    return R_tmp
#奇异值分解（SVD），特征降维方法
from sklearn.decomposition import TruncatedSVD
def SVD(Umatrix,x,inp='none'):
    R_tmp = copy.copy(Umatrix)
    R_tmp = R_tmp.astype(float)
    if inp!='none':
        R_tmp = imputation(inp,Umatrix)#缺失值处理
    means = np.array([R_tmp[i][R_tmp[i]>0].mean()for i in range(len(R_tmp))]).reshape(-1,1)
    R_tmp = R_tmp-means   #进行奇异值分解之前将数值减去该行的平均值，是算法效果更好
    svd = TruncatedSVD(n_components=K,random_state=4)
    R_k = svd.fit_transform(R_tmp)
    R_tmp = svd.inverse_transform(R_k)
    R_tmp = means+R_tmp
    return np.round(R_tmp,0)
def SVD_gm(Umatrix,K,inp='none',iterations=50,tol=0.01):
    R_tmp = copy.copy(Umatrix)
    R_tmp = R_tmp.astype(float)
    nrows = len(Umatrix)
    ncols = len(Umatrix[0])
    if inp!='none':
        R_tmp = imputation(inp,Umatrix)#缺失值处理
    
    svd = TruncatedSVD(n_cimponents=K,random_state=4)
    err = -1
    for it in range(iterations):
        R_k = svd.fit_transform(R_tmp)
        R_tmp = svd.inverse_transform(R_k) 
        err = 0
        for i in range(nrows):
            for j in range(ncols):
                if Umatrix[i][j]>0:
                    err+=pow(Umatrix[i][j]-R_tmp[i][j],2)
                    R_tmp[i][j] = Umatrix[i][j]
        if err<tol:
            print(it,'toll reached')
            break
    return np.round(R_tmp,0) 
        

In [43]:
#5.4 CBF方法（基于内容的过滤方法）
#从描述商品的数据中抽取用户的特征

#构造二进制向量，并将其置于DataFrame对象中
movieslist = [int(m.split(';')[-1]) for m in dfout.columns[1:]]
moviescats = ['unknown','Action','Adventure','Animation','Children\'s','Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western']
dfout_movies =  pd.DataFrame(columns=['movie_id']+moviescats)
startcatsindx = 5
cnt= 0
for m in movieslist:
    dfout_movies.loc[cnt] = [m]+df_info.iloc[m-1][startcatsindx:].tolist()
    cnt +=1 
print(dfout_movies.head())

dfout_movies.to_csv('./data/movies_content.csv',index=None)

#5.4.1商品特征平均得分方法
class CBF_averageprofile(object):
    def __init__(self,Movies,Movieslist):
        self.nfeatures = len(Movies[0])
        self.Movieslist = Movieslist
        saelf.Movies = Movies
    def GetRecMovies(self,u_vec,indxs=False):   #构造函数生成用户喜欢的电影类型向量，返回与该向量最相似的电影
        nmovies = len(u_vec)
        nfeatures = self.nfeatures
        mean_u = u_vec[u_vec>0].mean()
        diff_u = u_vec-mean_u
        features_u = np.zeros(nfeatures).astype(float)
        cnts = np.zeros(nfeatures)
        for m in range(nmovies):         #计算电影书，为下一步计算均值做准备
            if cnts[m]>0:
                features_u+= self.Movies[m]*(diff_u[m])
                cnts += self.Movies[m]
        for m in range(nfeatures):        #计算均值
            if cnts[m]>0:
                features_u[m] = features_u[m]/float(cnts[m])
        sims = np.zeros(nmovies)         #计算相似度
        for m in range(nmovies):
            if u_vec[m] == 0:
                sims[m] = sim(features_u,self.Movies[m])
        order_Movies_indxs = np.argsort(sims)[::-1]
        if indxs:
            return order_movies_indxs
        return self.Movieslist[order_movies_indxs]
    
 #5.4.2正则化线性回归方法
class CBF_regression(object):
    def __init__(self,Movies,Umatrix,alpha=0.01,l=0.0001,its=50,tol=0.001):
        self.nfeatures = len(Movies[0])+l
        nusers = len(Umatrix)
        nmovies = len(Umatrix[0])
        movies_feats = np.ones((nmovies,self.nfeatures))
        movies_feats[:,1:] = Movies
        saelf.movies_feats = movies_feats.astype(float)
        self.Umatrix = Umatrix.astype(float)          #使用梯度下降方法寻找参数sitar，即Pmatrix
        Pmatrix = np.random.rand(nusers,self.nfeatures)
        Pmatrix[:,0] = 1
        err=0
        cost = -1
        for it in range(its):
            print('its:',it,'--',cost)
        for u in range(nusers):
            for f in range(self.nfeatures):
                if f==0:
                    for m in range(nmovies):
                        if self.Umatrix[u,m]> 0:
                            diff = np.dot(Pmatrix[u],self.movies_feats[m])-self.Umatrix[u,m]
                            Pmatrix[u,f] += -alpha*(diff*self.movies_feats[m][f])
                else:
                    for m in range(nmovies):
                        if self.Umatrix[u,m]>0:
                            diff = np.dot(Pmatrix[u],self.movies_feats[m])-self.Umatrix[u,m]
                            Pmatrix[u,f] += -alpha*(diff*self.movies_feats[m][f]+l*Pmatrix[u][f])
        cost = 0
        for u in range(nusers):
            for m in range(nmovies):
                if self.Umatrix[u][m]>0:
                    cost+=0.5*pow(Umatrix[u][m]-np.dot(Pmatrix[u],self.movies_feats[m]),2)
            for f in range(1,self.nfeatures):
                cost += float(1/2.0)*pow((Pmatrix[u][f],2))
            if cost<tol:
                print("err",cost)
                break
        self.Pmatrix = Pmatrix
    def CalcRatings(self,u_vec):    #从效用矩阵R中与用户向量计算相似度量，找出最相似的分数向量
        s = 0.
        u_feats = np.zeros(len(self.Pmatrix[0]))
        
        for u in range(len(self.Umatrix)):
            tmps = sim(self.Umatrix[u],u_vec)
            if tmps>s:
                s = tmps
                u_feats = self.Pmatrix[u]
            if s == 1.:
                break
        new_vec = np.zeros(len(u_vec))
        for r in range(len(u_vec)):
            if u_vec[r]==0:
                new_vec[r] = np.dot(u_feats,self.movies_feats[r])
        return new_vec
#使用关联规则学习，构建推荐系统
#主要思想：统计交易数据库T中商品的出现情况，找到商品中的关系
#支持度（包含一组商品X的交易数据站总交易数据的比例）置信度（同时包含XY的交易数据占只包含X的比例）

class AssociationRules(object):
    def __init__(self,Umatrix,Movieslist,min_support=0.1,min_confidence=0.1,likethreshold=3):
        self.min_support = min_support   #支持度阈值
        self.Movieslist = Movieslist
        nitems = len(Umatrix[0])
        transactions = []
        for u in Umatrix:
            s = [i for i in range(len(u)) if u[i]>likethreshold]
            if len(s)>0:
                transactions.append(s)
        flat = [item for sublist in transactions for item in sublist]
        inititems = map(frozenset,[[item] for item in frozenset(flat)])
        set_trans = map(set,transactions)
        sets_init,self.dict_sets_support = self.filterSet(set_trans,inititems)
        setlen = 2
        item_tmp = self.combine_lists(sets_init,setlen)
        self.freq_sets,suo_tmp = self.filterSet(set_trans,item_tmp)
        self.ass_matrix = np.zeros(nitems,nitems)
        for freqset in self.freq_sets:
            list_setitems = [frozenset([item])for item in freqset]
            self.calc_confidence_matrix(freqset,list_setitems)
            
    def filterSet(self,set_trans,likeditems):  #用于过滤规则
        itemscnt = {}
        for id in set_trans:
            for item in likeditems:
                if item.issunset(id):
                    itemscnt.setdefault(item,0)
                    itemscnt[item] += 1
        num_items = float(len(set_trans))
        freq_sets = []
        dict_sets = {}
        for key in itemscnt:
            support = itemcnt[key]/num_items
            if support>=self.min_support:
                freq_sets.insert(0,key)
            dict_sets[key] = support
        return freq_sets,dict_sets
    def combine_lists(self,freq_sets,setlen):  #寻找所有可能的规则
        setitems_list = []
        nsets = len(freq_sets)
        for i in range(nsets):
            setlist1 = list(freq_sets[i])[:setlen-2]
            setlist2 = list(freq_sets[j])[:setlen-2]
            if set(setlist)==set(setlist2):
                setitems_list.append(freq_sets[i].union(freq_sets[j]))
        return set_items_list
    def calc_confidence_matrix(self,freqset,list_setitems):
        for target in list_setitems:#用满足最小阈值的置信度填充ass_matrix矩阵
            confidence = self.dict_sets_support[freqset]/self.dict_sets_support[freqsets-target]
            if confidence >= self.min_confidence:
                self.ass_matrix[list(freqset-target)[0]][list(target)[0]]=confidence
    
    def GetRecItems(self,u_vec,indxs=False):   #根据用户打的分数u_vec返回一个电影推荐列表
        vec_recs = np.dot(u_vec,self.ass_matrix)
        sortedweight = np.argsort(vec_recs)
        seenindxs = [indx for indx in range(len(u_vec))if u_vec[indx]>0]
        seenmovies = np.array(self.Movieslist)[seenindx]
        recitems = np.array(self.Movieslist)[sortedweight]
        recitems = [m for m in recitems if m not in seenmovies]
        if indx:
            vec_recs[seenindxs] = -1
            recsvec = np.argsort(vec_recs)[::-1][np.argsort(vec_recs)>0]
            return recsvec
        return recitems[::-1]
#5.6对数似然比推荐方法
#对数似然比（LLr）是度量事件AB不太可能是独立事件而是共现几率大于偶然的一种方法
class LogLikelihood(object):
    def __init__(self,Umatrix,Movieslist,likethreshold=3):
        self.Movieslist = Movieslist
        self.nusers = len(Umatrix)
        self.Umatrix = Umatrix
        self.likethreshold = likethreshold
        self.likerage = range(self.likethreshold+1,5+1)
        self.dislikerange = range(1,self.likethreshold)
        self.loglikelihood_ratio()
        
    def calc_k(self,a,b):
        tmpk = [[0 for j in range(2)]for i in range(2)]#构造二项分布矩阵K
        for ratings in self.Umatrix:
            if ratings[a] in self.likerage and ratings[b] in self.likerage:
                tmpk[0][0]+=1
            if ratings[a] in self.likerage and ratings[b] in self.dislikerange:
                tmpk[0][1]+=1
            if ratings[a] in self.dislikerange and ratings[b] in self.likerage:
                tmpk[1][0]+=1
            if ratings[a] in self.dislikerange and ratings[b] in self.dislikerange:
                tmpk[1][1]+=1
        return tmpk
    def calc_llr(self,k_matrix):
        Hcols = Hrows = Htot = 0.0
        if sum(k_matrix[0])+sum(k_matrix[1])==0:
            return 0
        invN = 1.0/(sum(k_matrix[0])+sum(k_matrix[1]))
        for i in range(0,2):                     
            if((k_matrix[0][i]+k_matrix[1][i])!=0.0):
                Hcols += invN*(k_matrix[0][i]+k_matrix[1][i])*math.log((k_matrix[0][i]+k_matrix[1][i])*invN)
            if((k_matrix[i][0]+k_matrix[i][1])!=0.0):
                Hrows += invN*(k_matrix[i][0]+k_matrix[i][1])*math.log((k_matrix[i][0]+k_matrix[i][1])*invN)
            for j in range(0,2):
                if(k_matrix[i][j]!=0.0):
                    Htot += invN*(k_matrix[i][j]*math.log(invN*k_matrix[i][j]))
        return 2.0*(Htot-Hcols-Hrows)/invN  #返回根据LLR计算公式得到的LLR值
    def loglikelihood_ratio(self):   #用于计算矩阵K（即该函数中变量calc_k）以及相应的LLR（calc_llr）
        nitems = len(self.Movieslist)
        self.items_llr = pd.DataFrame(np.zeros((nitems,nitems))).astype(float)
        for i in range(nitems):
            for j in range(nitems):
                if(j>=i):
                    tmpk= self.calc_k(i,j)
                    self.items_llr.loc[i,j] = self.calc_llr(tmpk)
                else:
                    self.items_llr.loc[i,j] = self.items_llr.iat[j,i]
    def GetRecItems(self,u_vec,indxs=False):
        items_weight = np.dot(u_vec,self.items_llr)
        sortedweight = np.argsort(items_weight)
        seenindxs = [indx for indx in range(len(u_vec))if u_vec[indx]>0]
        seenmovies = np.array(self.Movieslist)[seenindx]
        recitems = np.array(self.Movieslist)[sortedweight]
        recitems = [m for m in recitems if m not in seenmovies]
        if indx:
            items_weight[seenindxs] = -1
            recsvec = np.argsort(items_weight)[::-1][np.argsort(vec_recs)>0]
            return recsvec
        return recitems[::-1]
    


  movie_id unknown Action Adventure Animation Children's Comedy Crime  \
0        1       0      0         0         1          1      1     0   
1        2       0      1         1         0          0      0     0   
2        3       0      0         0         0          0      0     0   
3        4       0      1         0         0          0      1     0   
4        5       0      0         0         0          0      0     1   

  Documentary Drama Fantasy Film-Noir Horror Musical Mystery Romance Sci-Fi  \
0           0     0       0         0      0       0       0       0      0   
1           0     0       0         0      0       0       0       0      0   
2           0     0       0         0      0       0       0       0      0   
3           0     1       0         0      0       0       0       0      0   
4           0     1       0         0      0       0       0       0      0   

  Thriller War Western  
0        0   0       0  
1        1   0       0  
2        1 

In [46]:
#5.7混合推荐系统
#混合使用CBF和CF方法
#加权（CBF和CF预测分数取加权平均值）混合（CF与CBF预测结果合并为一个列表）
#切换（根据特地特定规则选择使用）特征组合（综合CF与CBF特征找到最相似的用户或商品）
#特阵扩充（类似于特征组合+附加特征预测分数）

#方法一：基于用户的CF，扩展效用矩阵，增加每位用户的平均分
class Hybird_cbf_cf(object):
    def __init__(self,Movies,Movieslist,Umatrix):
        self.nfeatures = len(Movies[0])
        self.movieslist = Movieslist
        self.Movies = Movies.astype(float)
        self.Umatrix_mfeats = np.zeros((len(Umatrix),len(Umatrix[0])+self.nfeatures))
        means = np.array([Umatrix[i][Umatrix[i]>0].mean()for i in range(len(Umatrix))]).reshape(-1,1)
        diffs = np.array([[Umatrix[i][j]-means[i] if Umatrix[i][j]>0 else 0.
                          for j in range(len(Umatrix[i]))]for i in range(len(Umatrix))])
        self.Umatrix_mfeats[:,:len(Umatrix[0])] = Umatrix
        self.nmovies = len(Movies)
        for u in range(len(Umatrix)):
            u_vec = Umatrix[u]
            self.Umatrix_mfeats[u,len(Umatrix[0]):] = self.GetUserItemFeatures(u_vec)
    def GetUserItemFeatures(self,u_vec):
        mean_u = u_vec[u_vec>0].mean()
        features_u = np.zeros(self.nfeatures).astype(float)
        cnts = np.zeros(self.nfeatures)
        for m in range(self.nmovies):
            if u_vec[m]>0:
                features_u += self.Movies[m]*u_vec[m]
                cnts+=self.Movies[m]
        for m in range(self.nfeatures):
            if cnts[m]>0:
                features_u[m]=features_u[m]/float(cnts[m])
        return features_u
    
    def CalcRating(u_vec,r,neighs):  #使用皮尔逊相关系数度量方法，比较每一位用户扩展后的特征向量
        rating=0.
        den=0.
        for j in range(len(neighs)):
            rating+=neighs[j][-1]*float(neighs[j][r]-neighs[j][neighs[j]>0][:-1].mean())
            den+=abs(neighs[j][-1])
        if den>0:
            rating = np.round(u_vec[u_vec>0].mean()+[rating/den],0)
        else:
            rating = np.round(u_vec[u_vec>0].mean())
        if rating>5:
            return 5
        elif rating<1:
            return 1
        return rating
    nrows = len(self.Umatrix_mfeats)
    ncols = len(self.Umatrix_mfeats[0])
    data_sim = np.zeros((nrows,ncols+1))
    data_sim[:,:-1] = self.Umatrix_mfeats
    u_rec = np.zeros(len(u_vec))
    mean = u_vec[u_vec>0].mean()
    u_vec_feats = u_vec
    u_vec_feats = np.append(u_vec_feats,self.GetUserItemFeatures(u_vec))
    for u in range(nrows):
        if np.array_equal(data_sim[u,:-1],u_vec)==False:
            data_sim[u,ncols] = sim(data_sim[u,:-1],u_vec_feats)
        else:
            data_sim[u,ncols] = 0
    data_sim[:,:-1] = self.Umatrix_mfeats
    u_rec = np.zeros(len(u_vec))
    mean = u_vec[u_vec>0].mean()
    u_vec_feats = u_vec
    u_vec_feats = np.append(u_vec_feats,self.GetUserItemFeatures(u_vec))

    for u in range(nrows):
        if np.array_equal(data_sim[u,:-1],u_vec) == False:
            data_sim[u,ncols] = sim(data_sim[u,:-1],u_vec_feats)
        else:
            data_sim[u,ncols] = 0

    data_sim = data_sim[data_sim[:,ncols].argsort()][::-1]

    for r in range(self.nmovies):#调用K近邻算法找出K个近邻
        if u_vec[r]==0:
            neighs = FindKNeighbours(r,data_sim,K)
            u_rec[r] = CalcRatings(u_vec,r,neighs)
    return u_rec
#  方法二：使用SVD（奇异值分解）方法扩展效用矩阵
class Hybird_svd(object):
    def __init__(self,Movies,Movieslist,Umatrix,K,inp):
        self.nfeatures = len(Movies)
        self.Movieslist = Movieslist
        self.Movies = Movies.astype(float)
        R_tmp = copy.copy(Umatrix)
        R_tmp = R_tmp.astype(float)
        if inp!='none':
            R_tmp = imputation(inp,Umatrix)#缺失值处理
        means = np.array([Umatrix[i][Umatrix[i]>0].mean()for i in range(len(Umatrix))]).reshape(-1,1)
        diffs = np.array([[float(Umatrix[i][j]-means[i])
                          if Umatrix[i][j]>0 else float(R_tmp[i][j]-means[i])
                          for j in range(len(Umatrix[i]))]
                         for i in range(len(Umatrix))])
        Umatrix_mfeats[:,:len(Umatrix[0])] = diffs
        self.nmovies = len(Movies)
        for u in range(len(Umatrix)):
            u_vec = Umatrix[u]
            Umatrix_mfeats[u,len(Umatrix[0]):] = self.GetUserItemFeatures(u_vec)
        
        svd = TruncatedSVD(n_components=K,random_stats=4)
        R_k = svd.fit_transform(Umatrix_mfeats)
        R_tmp = means+svd.inverse_transform(R_k)
        self.matrix = np.round(R_tmp[:,:self.nmovies],0)
    def GetUserItemFeatures(self,u_vec):
        mean_u = u_vec[u_vec>0].mean()
        diff_u = u_vec-mean_u
        features_u = np.zeros(self.nfeatures).astype(float)
        cnts = np.zeros(self.nfeatures)
        for m in range(self.nmovies):
            if u_vec[m]>0:
                features_u += self.Movies[m]*(diff_u[m])
                cnts+=selfself.Movies[m]
        for m in range(self.nfeatures):
            if cnts[m]>0:
                features_u[m] = features_u[m]/float(vnts[m])
        return features_u
    

SyntaxError: 'return' outside function (Temp/ipykernel_37436/2824200769.py, line 81)

In [57]:
#5.8推荐系统评估
#使用K这交叉检验验证结果的客观性
def cross_validation(df,k):
    val_num = int(len(df)/float(k))
    print(val_num)
    df_trains = []
    df_vals = []
    for i in range(k):
        start_val = (k-i-1)*val_num
        end_val = start_val+val_num
        df_trains.append(pd.concat([df[:start_val],df[end_val:]]))
        df_vals.append(df[start_val:end_val])
    return df_trains,df_vals
import random
def HideRandomRatings(u_vec,ratiovals=0.5):
    u_test = np.zeros(len(u_vec))
    u_vals = np.zeros(len(u_vec))
    cnt = 0
    nratings = len(u_vec[u_vec>0])
    for i in range(len(u_vec)):
        if u_vec[i]>0:
            if bool(random.getrandbits(1))or cnt>=int(nratings*ratiovals):
                u_test[i] = u_vec[i]
            else:
                cnt+=1
                u_vals[i] = u_vec[i]
    return u_test,u_vals  #u_vals存储的是预测值，u_test存储用于测试算法的实际分数
#读取数据
df = pd.read_csv('./data/utilitymatrix.csv')
print(df.head())
df_movies = pd.read_csv('./data/movies_content.csv')
movies = df_movies.values[:,1:]
print('check:::',len(df.columns[1:]),'--',len(df_movies))
movieslist = list(df.columns[1:])
nfolds = 5            #调用自定义的K折检验函数
df_trains,df_vals = cross_validation(df,nfolds)

nmovies =  len(df_vals[0].values[:,1:][0])
s = []
tests_vecs_folds = []
vals_vecs_folds = []
for i in range(nfolds):
    u_vecs = df_vals[i].values[:,1:]
    vtests = np.empty((0,nmovies),float)
    vvals = np.empty((0,nmovies),float)
    for u_vec in u_vecs:
        u_test,u_vals = HideRandomRatings(u_vec)
        vvals = np.vstack([vvals,u_vals])
        vtetss = np.vstack([vtests,u_test])
    vals_vecs_folds.append(vvals)
    tests_vecs_folds.append(vtests)

   user  Toy Story (1995);1  GoldenEye (1995);2  Four Rooms (1995);3  \
0     1                   5                   3                    4   
1     2                   4                   0                    0   
2     3                   0                   0                    0   
3     4                   0                   0                    0   
4     5                   4                   3                    0   

   Get Shorty (1995);4  Copycat (1995);5  Twelve Monkeys (1995);7  \
0                    3                 3                        4   
1                    0                 0                        0   
2                    0                 0                        0   
3                    0                 0                        0   
4                    0                 0                        0   

   Babe (1995);8  Dead Man Walking (1995);9  Richard III (1995);10  ...  \
0              1                          5                      3  ...   
1 

In [58]:
#5.8.1均方根误差评估（RMSE）（支持用于CF和线性回归CBF）
def SE(u_preds,u_vals):
    nratings = len(u_vals)
    se = 0
    cnt = 0
    for i in range(nratings):
        if u_vals[i]>0:
            se += (u_vals[i]-u_preds[i])*(u_vals[i]-u_preds[i])
            cnt+=1
    return se,cnt

err_itembased = 0.
cnt_itembased = 0
err_userbased = 0.
cnt_userbased = 0
err_slopeone = 0.
cnt_slopeone = 0
err_cbfcf = 0.
cnt_cbfcf = 0
for i in range(nfolds):
    Umatrix = df_trains[i].values[:,1:]
    cfitembased = CF_itembased(Umatrix)
    cfslopeone = SlopeOne(Umatrix)
    print('fold:',i+1)
    vec_vals = vals_vecs_folds[i]
    vec_tests = tests_vecs_folds[i]
    for j in range(len(vec_vals)):
        u_vals = vec_vals[j]
        u_test = vec_tests[j]
        
        u_preds = cbfcf.CalcRatings(u_test,5)
        e,c = SE(u_preds,u_vals)
        err_cbfcf += e
        cnt_cbfcf += c
        
        u_preds = CF_userbased(u_test,5,Umatrix)
        e,c = SE(u_preds,u_vals)
        err_userbased += e
        cnt_userbased += c
        
        u_preds = cfitembased.CalcRatings(u_test,5)
        e,c = SE(u_preds,u_vals)
        err_itembased += e
        cnt_itembased += c
        
        u_pred = cfslopeone.CalcRatings(u_test,5)
        e,c = SE(u_preds,u_vals)
        err_slopeone += e
        cnt_slopeone += c
rmse_userbased = np.sqrt(err_userbased/float(cnt_userbased))
rmse_itembased = np.sqrt(err_itembased/float(cnt_itembased))
rmse_slopeone = np.sqrt(err_slopeone/float(cnt_slopeone))
print('user_userbased rmse:',rmse_userbased,'--',cnt_userbased)
print('user_itembased rmse:',rmse_itembased,'--',cnt_itembased)
print('slope one rmse:',rmse_slopeone,'--',cnt_slopeone)

rmse_cbfcf = np.sqrt(err_cbfcf/float(cnt_cbfcf))
print('cbfcf rmse',rmse_cbfcf,'--',cnt_cbfcf)


TypeError: CF_itembased() takes no arguments

In [59]:
#8.2分类效果的度量方法
def ClassificationMetrics(vec_vals,vec_recs,likethreshold=3,shortlist=50,ratingsval=False,vec_test=None):
    indxs_like = [i for i in range(len(vec_vals))if vec_vals[i]>likethreshold]
    indxs_dislike = [i for i in range(len(vec_vals))if vec_vals[i]<=likethreshold and vec_vals[i]>0]
    cnt = len(indxs_like)+len(indxs_dislike)
    indxs_rec = []
    if ratingsval:
        if vec_test == None:
            raise('Error no test vector')
            indxs_rec = [i for i in range(len(vec_recs))if vec_recs[i]>likethreshold and vec_test[i]<1][:shortlist]
        else:
            indxs_rec = vec_recs[:shortlist]
        tp = len(set(indxs_rec).intersection(set(indxs_like)))
        fp = len(set(indxs_rec).intersection(set(indxs_dislike)))
        fn = len(set(indxs_like)^(set(indxs_rec).intersection(set(indxs_like))))
        precision = 0.
        if tp+fp>0:
            precision = float(tp)/(tp+fp)
        recall = 0
        if tp+fn>0:
            recall = float(tp)/(tp+fn)
        f1 = 0
        if recall_precision>0:
            f1 = 0.*precision*recall/(precision+recall)
        return ap.array([precision,recall,f1]).cat